In [96]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta

In [62]:
# Função para obter a cotação do dólar na data desejada
def obter_cotacao(data):
    # Converter a data para o formato americano (MM-DD-YYYY)
    data_formatada = data.strftime("%m-%d-%Y")
    url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/" \
          f"CotacaoDolarDia(dataCotacao=@dataCotacao)?@dataCotacao='{data_formatada}'&$format=json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        dados = response.json()
        if len(dados['value']) > 0:
            return dados['value'][0]['cotacaoCompra']
        else:
            return None  # Sem cotação para o dia
    except Exception as e:
        print(f"Erro ao obter cotação para {data}: {e}")
        return None

In [63]:
# Função para buscar cotação válida (dia útil mais próximo, se necessário)
def buscar_cotacao_valida(data):
    tentativa = data
    while tentativa.weekday() in (5, 6) or obter_cotacao(tentativa) is None:
        tentativa -= timedelta(days=1)  # Retroceder para o dia útil anterior
    return obter_cotacao(tentativa)

In [64]:
df = pd.read_csv(r'C:\Users\GabrielCielo\Desktop\consolatio\Databases\profitDBcomHora.csv')

produtos = ['Camisa X-Tretch', 'Camisa Ultra-Stretch', 'Camisa Polo Ultra', 'Calça X-Tretch',
            'Camisa X-Tretch Manga Curta', 'Kit Cuecas 5', 'Kit Cuecas 10',
            'Necessaire', 'Troca estendida (30 dias)']

mapeamento_nomes = {
    'Camisa Consolatio Ultra-Stretch':'Camisa Ultra-Stretch',
    'Camisa Consolatio X-Tretch':'Camisa X-Tretch',
    'Calça Social X-Tretch':'Calça X-Tretch',
    'Kit Cuecas Boxer Respiráveis':'Kit Cuecas 5',
    'Camisa Ultra-Stretch | Especial de Natal':'Camisa Ultra-Stretch',
    'Camisa X-Tretch | Especial de Natal':'Camisa X-Tretch',
    'Kit Cuecas Boxer Respiráveis (Compre 3, Leve 5)':'Kit Cuecas 5',
    'Especial de Natal - Camisa X-Tretch':'Camisa X-Tretch',
}

df['product_title'] = df['product_title'].replace(mapeamento_nomes)

df_filtrado = df[df['product_title'].isin(produtos)]

df_filtrado = df_filtrado.copy()

custos_predefinidos = {
    'Camisa X-Tretch': {1:15.16 , 2:24.59/2 , 3:33.71/3, 4:49.29/4, 5:59.93/5},
    'Camisa Ultra-Stretch': {1:18.13 , 2:31.36/2 , 3:44.19/3, 4:63.17/4, 5:82.02/5},
    'Camisa Polo Ultra': {1:12.19 , 2:20.51/2, 3:29.03/3, 4:37.03/4, 5:44.36/5},
    'Calça X-Tretch': {1:13.61, 2:23.05/2, 3:33.02/3, 4:40.46/4, 5:49.61/5},
    'Camisa X-Tretch Manga Curta': {1:18.5},
    'Kit Cuecas 5': {1:10.19},
}

In [65]:
def obter_custo_por_produto(produto, quantidade):
    if produto in custos_predefinidos:
        custos = custos_predefinidos[produto]
        # Encontrar a maior chave menor ou igual à quantidade
        max_quantidade = max((q for q in custos.keys() if q <= quantidade), default=None)
        if max_quantidade is not None:
            return custos[max_quantidade]
    return None  # Retorna None se não encontrar correspondência

In [66]:
# Aplicar a função para calcular o custo
df_filtrado['Custo_Unitario_USD'] = df_filtrado.apply(lambda row: obter_custo_por_produto(row['product_title'], row['net_quantity']), axis=1)


In [67]:
df_filtrado['Custo_USD'] = df_filtrado['Custo_Unitario_USD'] * df_filtrado['net_quantity']

df_filtrado['day'] = pd.to_datetime(df_filtrado['day'])

In [68]:
import time

inicio = time.time()

df_filtrado['Cotacao_USD_BRL'] = df_filtrado['day'].apply(buscar_cotacao_valida)

fim = time.time()

tempo_total = fim - inicio
print(f"O script demorou {tempo_total:.2f} segundos para executar.")

O script demorou 10009.23 segundos para executar.


In [85]:
df_filtrado['Custo_USD'] = df_filtrado['Custo_USD'].fillna(0)

df_filtrado['Custo_Unitario_USD'] = df_filtrado['Custo_Unitario_USD'].fillna(0)

df_filtrado['Custo_BRL'] = df_filtrado['Cotacao_USD_BRL'] * df_filtrado['Custo_USD']

df_filtrado['Margem_Bruta'] = df_filtrado['net_sales'] - df_filtrado['Custo_BRL']




In [108]:
df_filtrado['Margem_Lucro_Pct'] = np.where(
    df['net_sales'] == 0,
    1, # Se receita for 0, margem de lucro percentual será 100%
    (df['Margem_Bruta'] / df['net_sales']) 
)

In [109]:
df = df_filtrado.copy()

In [110]:
# Salvar o DataFrame em um arquivo CSV
df.to_csv(r'C:\Users\GabrielCielo\Desktop\consolatio\Databases\ProfitDB.csv', index=False)

In [103]:
df.head()

,product_title,variant_title,variant_sku,order_name,order_id,hour_of_day,day,net_quantity,gross_sales,discounts,...,net_sales,taxes,total_sales,Custo_Unitario_USD,Custo_USD,Cotacao_USD_BRL,Custo_BRL,Lucro_Bruto,Margem_Bruta,Margem_Lucro_Pct
0,Camisa Ultra-Stretch,Preto / PP,16237997378-CAUS-PRET-0PP,#623303,5952991133920,10,2025-01-09,9,2699.1,-993.34,...,1705.76,0.0,1705.76,16.404,147.636,6.0890,898.955604,806.804396,806.804396,0.472988
1,Camisa Ultra-Stretch,Branco / GG,16237997369-CAUS-BRAN-0GG,#548573,5858893496544,18,2024-11-04,6,1739.4,-214.38,...,1525.02,0.0,1525.02,16.404,98.424,5.7892,569.796221,955.223779,955.223779,0.626368
2,Camisa X-Tretch,Azul Marinho / GG,16237998663-CAXT-AMNH-0GG,#601263,5916916023520,9,2024-12-12,8,1519.2,-130.91,...,1388.29,0.0,1388.29,11.986,95.888,5.9402,569.593898,818.696102,818.696102,0.589715
3,Camisa X-Tretch,Branco / M,16237998580-CAXT-BRAN-00M,#580193,5892645617888,12,2024-11-28,7,1329.3,-369.90,...,959.40,0.0,959.40,11.986,83.902,5.9865,502.279323,457.120677,457.120677,0.476465
4,Camisa Ultra-Stretch,Branco / P,16237997363-CAUS-BRAN-00P,#598883,5913934004448,8,2024-12-10,5,1449.5,-577.08,...,872.42,0.0,872.42,16.404,82.020,6.0523,496.409646,376.010354,376.010354,0.430997


In [116]:
df[df['day'] == 0]

,product_title,variant_title,variant_sku,order_name,order_id,hour_of_day,day,net_quantity,gross_sales,discounts,...,net_sales,taxes,total_sales,Custo_Unitario_USD,Custo_USD,Cotacao_USD_BRL,Custo_BRL,Lucro_Bruto,Margem_Bruta,Margem_Lucro_Pct
17404,Camisa X-Tretch,Branco / G,16237998584-CAXT-BRAN-00G,#622363,5951856509152,8,2025-01-08,1,199.9,-199.9,...,0.0,0.0,0.0,15.160,15.16,6.1315,92.953540,-92.953540,-92.953540,1.0
17405,Camisa X-Tretch,Azul Claro / GG,16237998618-CAXT-ACLA-0GG,#571083,5882416431328,17,2024-11-21,2,379.8,-379.8,...,0.0,0.0,0.0,12.295,24.59,5.8167,143.032653,-143.032653,-143.032653,1.0
17406,Camisa X-Tretch,Azul Claro / XGG,16237998621-CAXT-ACLA-XGG,#610703,5933338001632,10,2024-12-23,1,199.9,-199.9,...,0.0,0.0,0.0,15.160,15.16,6.1606,93.394696,-93.394696,-93.394696,1.0
17407,Camisa Ultra-Stretch,Grafite / M,16349791208-CAUS-GRAF-00M,#611953,5935264039136,17,2024-12-24,1,299.9,-299.9,...,0.0,0.0,0.0,18.130,18.13,6.1527,111.548451,-111.548451,-111.548451,1.0
17408,Camisa X-Tretch,Azul Claro / G,16237998617-CAXT-ACLA-00G,#579193,5891716481248,18,2024-11-27,1,189.9,-189.9,...,0.0,0.0,0.0,15.160,15.16,5.8285,88.360060,-88.360060,-88.360060,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20236,Camisa X-Tretch,Azul Claro / M,16237998615-CAXT-ACLA-00M,#617163,5942988079328,10,2024-12-31,1,199.9,-199.9,...,0.0,0.0,0.0,15.160,15.16,6.1917,93.866172,-93.866172,-93.866172,1.0
20237,Camisa X-Tretch,Preto / P,16237998593-CAXT-PRET-00P,#594503,5907374604512,8,2024-12-06,1,189.9,-189.9,...,0.0,0.0,0.0,15.160,15.16,6.0287,91.395092,-91.395092,-91.395092,1.0
20238,Camisa Ultra-Stretch,Grafite / M,16349791208-CAUS-GRAF-00M,#614113,5938062295264,11,2024-12-27,1,299.9,-299.9,...,0.0,0.0,0.0,18.130,18.13,6.1985,112.378805,-112.378805,-112.378805,1.0
20239,Camisa Ultra-Stretch,Grafite / GG,16349791211-CAUS-GRAF-0GG,#623713,5953295319264,15,2025-01-09,1,299.9,-299.9,...,0.0,0.0,0.0,18.130,18.13,6.0890,110.393570,-110.393570,-110.393570,1.0


In [74]:
#df_original = pd.read_csv(r'C:\Users\GabrielCielo\Desktop\consolatio\Databases\profitDBAtualizado.csv')

In [75]:
#df_original = df_original.drop(columns='product_price')

In [76]:
#df_original.shape[1] == df.shape[1] # conferindo se o número de colunas é o mesmo

In [77]:
#df_original.columns == df.columns

In [78]:
#df_atualizado = pd.concat([df_original, df],ignore_index=True) # adicionando novos dados a database original


In [79]:
#df_atualizado.to_csv(r'C:\Users\GabrielCielo\Desktop\consolatio\Databases\profitDBAtualizado.csv',index=False) # atualizando o arquivo csv

In [117]:
df_teste = df.copy()

In [119]:
df_teste = df_teste.loc[df_teste['net_sales'] != 0]

In [123]:
df_teste = df_teste.loc[df_teste['net_sales'] <0]

In [131]:
df_teste[df_teste['returns'] != 0]

,product_title,variant_title,variant_sku,order_name,order_id,hour_of_day,day,net_quantity,gross_sales,discounts,...,net_sales,taxes,total_sales,Custo_Unitario_USD,Custo_USD,Cotacao_USD_BRL,Custo_BRL,Lucro_Bruto,Margem_Bruta,Margem_Lucro_Pct


In [135]:
df_teste[df_teste['Margem_Lucro_Pct'] == -874.29292]

,product_title,variant_title,variant_sku,order_name,order_id,hour_of_day,day,net_quantity,gross_sales,discounts,...,net_sales,taxes,total_sales,Custo_Unitario_USD,Custo_USD,Cotacao_USD_BRL,Custo_BRL,Lucro_Bruto,Margem_Bruta,Margem_Lucro_Pct
17401,Camisa X-Tretch,Branco / G,16237998584-CAXT-BRAN-00G,#568343,5879672176864,12,2024-11-19,1,189.9,-189.8,...,0.1,0.0,0.1,15.16,15.16,5.7737,87.529292,-87.429292,-87.429292,-874.29292


In [138]:
df_teste[df_teste['net_sales'] < 10]

,product_title,variant_title,variant_sku,order_name,order_id,hour_of_day,day,net_quantity,gross_sales,discounts,...,net_sales,taxes,total_sales,Custo_Unitario_USD,Custo_USD,Cotacao_USD_BRL,Custo_BRL,Lucro_Bruto,Margem_Bruta,Margem_Lucro_Pct
16292,Camisa Ultra-Stretch,Grafite / G,16349791209-CAUS-GRAF-00G,#570773,5882148552928,13,2024-11-21,1,289.9,-280.57,...,9.33,0.0,9.33,18.13,18.13,5.8167,105.456771,-96.126771,-96.126771,-10.302977
16293,Camisa Ultra-Stretch,Branco / G,16237997367-CAUS-BRAN-00G,#570773,5882148552928,13,2024-11-21,1,289.9,-280.57,...,9.33,0.0,9.33,18.13,18.13,5.8167,105.456771,-96.126771,-96.126771,-10.302977
16294,Camisa Ultra-Stretch,Azul Claro / G,16237997408-CAUS-ACLA-00G,#570773,5882148552928,13,2024-11-21,1,289.9,-280.57,...,9.33,0.0,9.33,18.13,18.13,5.8167,105.456771,-96.126771,-96.126771,-10.302977
16295,Camisa Ultra-Stretch,Preto / G,16237997386-CAUS-PRET-00G,#570773,5882148552928,13,2024-11-21,1,289.9,-280.57,...,9.33,0.0,9.33,18.13,18.13,5.8167,105.456771,-96.126771,-96.126771,-10.302977
16296,Camisa Ultra-Stretch,Azul Claro / P,16237997399-CAUS-ACLA-00P,#610893,5933733347552,15,2024-12-23,1,299.9,-290.62,...,9.28,0.0,9.28,18.13,18.13,6.1606,111.691678,-102.411678,-102.411678,-11.035741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17399,Troca estendida (30 dias),NaN,TR30-30D,#581163,5892685562080,12,2024-11-28,1,4.0,-3.88,...,0.12,0.0,0.12,0.00,0.00,5.9865,0.000000,0.120000,0.120000,1.000000
17400,Troca estendida (30 dias),NaN,TR30-30D,#618253,5945139232992,11,2025-01-02,1,4.0,-3.89,...,0.11,0.0,0.11,0.00,0.00,6.2080,0.000000,0.110000,0.110000,1.000000
17401,Camisa X-Tretch,Branco / G,16237998584-CAXT-BRAN-00G,#568343,5879672176864,12,2024-11-19,1,189.9,-189.80,...,0.10,0.0,0.10,15.16,15.16,5.7737,87.529292,-87.429292,-87.429292,-874.292920
17402,Camisa X-Tretch,Azul Claro / XGG,16237998621-CAXT-ACLA-XGG,#557763,5865642426592,10,2024-11-10,1,189.9,-189.80,...,0.10,0.0,0.10,15.16,15.16,5.7642,87.385272,-87.285272,-87.285272,-872.852720
